In [1]:
import numpy as np
import random

# Objective Functions
def sphere_function(X):
    return sum(x**2 for x in X)

def rastrigin_function(X):
    return sum(x**2 - 10 * np.cos(2 * np.pi * x) + 10 for x in X)

def six_hump_camel_function(X):
    x1, x2 = X
    return 4*x1**2 - 2.1*x1**4 + (x1**6)/3 + x1*x2 - 4*x2**2 + 4*x2**4

# Hill Climbing Variants
def simple_hill_climbing(func, bounds, n_iterations=1000, step_size=0.1):
    solution = [random.uniform(b[0], b[1]) for b in bounds]
    solution_eval = func(solution)

    for _ in range(n_iterations):
        # Generate a neighbor solution
        candidate = [solution[i] + random.uniform(-step_size, step_size) for i in range(len(bounds))]
        # Clip to bounds
        candidate = [max(min(candidate[i], bounds[i][1]), bounds[i][0]) for i in range(len(bounds))]
        candidate_eval = func(candidate)

        if candidate_eval < solution_eval:  # Accept improvement
            solution, solution_eval = candidate, candidate_eval

    return solution, solution_eval

def stochastic_hill_climbing(func, bounds, n_iterations=1000, step_size=0.1):
    solution = [random.uniform(b[0], b[1]) for b in bounds]
    solution_eval = func(solution)

    for _ in range(n_iterations):
        # Randomly pick a dimension to perturb
        i = random.randint(0, len(bounds) - 1)
        candidate = list(solution)
        candidate[i] += random.uniform(-step_size, step_size)
        candidate[i] = max(min(candidate[i], bounds[i][1]), bounds[i][0])  # Clip to bounds
        candidate_eval = func(candidate)

        if candidate_eval < solution_eval:
            solution, solution_eval = candidate, candidate_eval

    return solution, solution_eval






def steepest_ascent_hill_climbing(func, bounds, n_iterations=1000, step_size=0.1):
    solution = [random.uniform(b[0], b[1]) for b in bounds]
    solution_eval = func(solution)

    for _ in range(n_iterations):
        # Generate all neighbors
        neighbors = []
        for i in range(len(bounds)):
            candidate1 = list(solution)
            candidate2 = list(solution)
            candidate1[i] += step_size
            candidate2[i] -= step_size
            candidate1[i] = max(min(candidate1[i], bounds[i][1]), bounds[i][0])
            candidate2[i] = max(min(candidate2[i], bounds[i][1]), bounds[i][0])
            neighbors.append(candidate1)
            neighbors.append(candidate2)

        # Evaluate all neighbors and choose the best
        best_neighbor = None
        best_eval = float('inf')
        for neighbor in neighbors:
            neighbor_eval = func(neighbor)
            if neighbor_eval < best_eval:
                best_neighbor, best_eval = neighbor, neighbor_eval

        if best_eval < solution_eval:
            solution, solution_eval = best_neighbor, best_eval

    return solution, solution_eval

# Genetic Algorithm
def genetic_algorithm(func, bounds, population_size=20, generations=100, crossover_rate=0.8, mutation_rate=0.1):
    def create_individual():
        return [random.uniform(b[0], b[1]) for b in bounds]

    def mutate(individual):
        for i in range(len(bounds)):
            if random.random() < mutation_rate:
                individual[i] += random.uniform(-0.1, 0.1)
                individual[i] = max(min(individual[i], bounds[i][1]), bounds[i][0])

    def crossover(parent1, parent2):
        if random.random() < crossover_rate:
            point = random.randint(1, len(bounds) - 1)
            return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]
        return parent1, parent2

    def select_parent(population):
        total_fitness = sum(1 / (1 + func(ind)) for ind in population)
        pick = random.uniform(0, total_fitness)
        current = 0
        for ind in population:
            current += 1 / (1 + func(ind))
            if current > pick:
                return ind

    # Initialize population
    population = [create_individual() for _ in range(population_size)]

    for _ in range(generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parent(population)
            parent2 = select_parent(population)
            offspring1, offspring2 = crossover(parent1, parent2)
            mutate(offspring1)
            mutate(offspring2)
            new_population.extend([offspring1, offspring2])
        population = new_population

    best_individual = min(population, key=func)
    return best_individual, func(best_individual)

# Run and Test
bounds_f1 = [(-100, 100)] * 10
bounds_f9 = [(-5.12, 5.12)] * 10
bounds_f16 = [(-5, 5), (-5, 5)]

# Hill Climbing
solution, value = simple_hill_climbing(sphere_function, bounds_f1)
print("Simple Hill Climbing (F1):", solution, "Value:", value)

solution, value = stochastic_hill_climbing(rastrigin_function, bounds_f9)
print("Stochastic Hill Climbing (F9):", solution, "Value:", value)

solution, value = steepest_ascent_hill_climbing(six_hump_camel_function, bounds_f16)
print("Steepest Ascent Hill Climbing (F16):", solution, "Value:", value)

# Genetic Algorithm
solution, value = genetic_algorithm(sphere_function, bounds_f1)
print("Genetic Algorithm (F1):", solution, "Value:", value)

solution, value = genetic_algorithm(rastrigin_function, bounds_f9)
print("Genetic Algorithm (F9):", solution, "Value:", value)

solution, value = genetic_algorithm(six_hump_camel_function, bounds_f16)
print("Genetic Algorithm (F16):", solution, "Value:", value)

Simple Hill Climbing (F1): [34.32112855353378, 19.180164609855023, -7.908868990429641, -45.51459352912077, -30.557478959162015, -44.253999839961395, -30.503315125476465, 1.8556557178837707, 1.972617320115047, -17.232954814938523] Value: 7806.88467718589
Stochastic Hill Climbing (F9): [0.9953069123930613, -3.9799707964154365, 0.001710118828721456, 2.9827658311737775, 1.9885188638776534, -3.9821329635732754, 0.0004086088422876888, -2.984634456221639, 1.989913728816519, 0.0027234699443549892] Value: 58.706770133507575
Steepest Ascent Hill Climbing (F16): [0.05742069377696585, -0.6859847844035847] Value: -1.0227607021124738
Genetic Algorithm (F1): [-68.44459122476977, -31.188992437844828, -42.33893252740379, 35.89890057251878, -30.131198804249266, 20.527375340949998, 3.0467245568975123, -47.5666359098918, 26.088677863090815, -26.680099110904365] Value: 13732.308050380463
Genetic Algorithm (F9): [-1.9292912663461526, -0.9084649880682348, -2.9956362649993125, -2.1039733665799165, 3.826308863